# Supervisor Architecture

In this architecture, we add each agent to the graph as a node and also add a supervisor node, which decides which agents should be called next. We use conditional edges to route execution to the appropriate agent node based on the supervisor’s decision.

Let’s first see what the supervisor node looks like:

In [6]:
from typing import Literal
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel
from utils import graph_to_image

class SupervisorDecision(BaseModel):
    next: Literal["researcher", "coder", "FINISH"]

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21", model="gpt-4o", temperature=0)
model = model.with_structured_output(SupervisorDecision)

agents = ["researcher", "coder"]

system_prompt_part_1 = f"""You are a supervisor tasked with managing a 
conversation between the following workers: {agents}. Given the following user 
request, respond with the worker to act next. Each worker will perform a
task and respond with their results and status. When finished,
respond with FINISH."""

system_prompt_part_2 = f"""Given the conversation above, who should act next? Or 
    should we FINISH? Select one of: {', '.join(agents)}, FINISH"""

def supervisor(state):
    messages = [
        ("system", system_prompt_part_1),
        *state["messages"],
        ("system", system_prompt_part_2)
    ]
    return model.invoke(messages)

> **Note**
>
> The code in the supervisor prompt relies on the assumption that the names of your subagents are **self-explanatory and distinct**.
>
> For example, if agents were named generically as `agent_1` and `agent_2`,  
> the LLM would not have enough information to determine which one is appropriate for a given task.
>
> To improve selection accuracy, consider **modifying the prompt** to include a short **description**  
> of each agent's capabilities. This can guide the LLM when making routing decisions.

Now let’s look at how to integrate the supervisor node into a larger graph that includes two subagents:

- `researcher`
- `coder`

The purpose of this setup is to handle queries that:

- Can be answered by the **researcher** alone
- Can be answered by the **coder** alone
- May require **both agents in succession**

This example doesn’t include implementations of the `researcher` or `coder` nodes themselves.  
They could be any LangGraph graphs or nodes, depending on your specific use case.


In [4]:
from typing import Literal
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START

model = AzureChatOpenAI(azure_deployment="gpt-4o", api_version="2024-10-21", model="gpt-4o", temperature=0)

class AgentState(BaseModel):
    next: Literal["researcher","coder", "FINISH"]

def researcher(state: AgentState):
    response = model.invoke(...)
    return {"messages": [response]}

def coder(state: AgentState):
    response = model.invoke(...)
    return {"messages": [response]}

builder = StateGraph(AgentState)
builder.add_node(supervisor)
builder.add_node(researcher)
builder.add_node(coder)

builder.add_edge(START, "supervisor")
# route to one of the agents or exit based on the supervisor's decision
builder.add_conditional_edges("supervisor", lambda state: state["next"])
builder.add_edge("researcher", "supervisor")
builder.add_edge("coder", "supervisor")

supervisor = builder.compile()

**A Few Things to Notice**

In this example, both subagents (`researcher` and `coder`) can **see each other’s work**,  
since all progress is stored in a shared `messages` list.

However, this isn’t the only way to structure a multi-agent system.

---

**Alternative Agent Designs**

Each subagent can be more complex. For example:

- A subagent could be implemented as its **own LangGraph graph**.
- It could maintain **internal state** and return only a **summary** of its actions.
- This allows for modular, nested graphs with encapsulated logic.

---

**Routing Back to the Supervisor**

After each agent completes its task, the flow routes back to the **supervisor node**.  
The supervisor then decides whether:

- More work is needed
- Another agent should be called
- The interaction should end

---

**Flexible Routing Options**

This back-to-supervisor design is **not a hard requirement**.

You could instead allow **each agent** to decide whether its output is final.  
To support this:

- Replace the direct edge (e.g., from `researcher` to `supervisor`) with a **conditional edge**.
- This edge would check a **state key** updated by the agent to determine what happens next.

This design increases flexibility and reduces supervisor overhead for certain tasks.


Let's go back to the [main file](../README.md#patterns-to-make-the-most-of-llms).